In [10]:
# Importamos JSON
import json

# Guardamos las credenciales para consumir api de twitter
credentials = {
    "KEY": "c0Jlf7B9Ez56i6rsnBSeiBCii",
    "KEY_SECRET": "yuCAV2TdGrvitWBGQondppKaxNlQfnrc5krRCYvNpu3miY2gQQ",
    "TOKEN": "1279067386520440833-YLZvyZzPDZlQ3bGa5SH2fhugVPl4Cs",
    "ACCESS_SECRET": "DZuy6rzCJFZ4723jVl2gITgf2jCXr1cqlFRtQXFytrIKD"
}

credentials_PRO = {
    "KEY": "XqNe5tbaycDxrrIOVqHR1NoPk",
    "KEY_SECRET": "wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q",
    "TOKEN": "1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ",
    "ACCESS_SECRET": "rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM"
}

credentials
credentials_PRO

{'KEY': 'XqNe5tbaycDxrrIOVqHR1NoPk',
 'KEY_SECRET': 'wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q',
 'TOKEN': '1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ',
 'ACCESS_SECRET': 'rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM'}

In [11]:
# Intslamos Twythom
!pip install twython

In [13]:
# Importar TwyThon (Consumir api de twitter)
from twython import Twython

# Objeto crendeiales
tweets = Twython(credentials["KEY"], credentials["KEY_SECRET"])

# Palabra a consultar
word = "Colombia"
# Consulta
query = {
    'q': word,
    'result_type': 'popular',
    'count': 2,
    'lang': 'es'
}

query

{'q': 'Colombia', 'result_type': 'popular', 'count': 2, 'lang': 'es'}

In [14]:
consulta = tweets.search(**query)['statuses']
#consulta

In [15]:
import pandas as pd

df = pd.json_normalize(consulta)
df

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang
0,Mon Jul 06 21:10:26 +0000 2020,1280247761535995906,1280247761535995906,¿Qué busco? Díganle al dueño de @ELTIEMPO que ...,True,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,NaN,NaN,False,239.0,1092.0,False,False,False,es
1,Mon Jul 06 17:26:19 +0000 2020,1280191360444882944,1280191360444882944,Hay municipios en Colombia a los que sólo las ...,False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
## consultar api tiempo real
from twython import TwythonStreamer

## Se guardara en un archivo .csv
import csv

#procesar los datos con filtro
def process_tweet(tweet):
    # guardar hashtag / contenido / usuario / localizacion / edad / mirar mas
    d = {}
    d["hashtags"] = [hashtag["text"] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_location'] = tweet['user']['location']
    return d

# Heredando clase
#se le pasa las credenciales (probar siempre con credentials normales.)
class Streamer(TwythonStreamer):

    #Datos recibidos
    def on_success(self, data):
        tweet_data = process_tweet(data)
        self.save_tweet(tweet_data)

    #Error
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    #Guardar en un archivo el resultados
    def save_tweet(self, tweet):
        #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
        #with open(r'tweets.csv', 'a', encoding="utf-8") as file: 
        with open(r'work/slides/data/tweets.csv', 'a', encoding="utf-8") as file:
            write = csv.writer(file)
            write.writerow(list(tweet.values()))

In [67]:
# Intanciar
stream =  Streamer(credentials['KEY'], credentials['KEY_SECRET'], credentials['TOKEN'], credentials['ACCESS_SECRET'])

#Iniciar consulta en tiempo real
stream.statuses.filter(track="Colombia")
# Para poder para la consulta y luego ver los resultados debe parar el proceso manualmente, es decir interrumpirlo.
# dando click en el boton de interrupt IPython kernel

KeyboardInterrupt: 

In [68]:
# #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
# tweetsRealTime = pd.read_csv("tweets.csv")
tweetsRealTime = pd.read_csv("work/slides/data/tweets.csv")
tweetsRealTime.head()

,hashtags,text,user,location
0,['ENPolítica'],RT @ElNacionalWeb: Falleció el diputado Hernán...,roswitha76,"ï¿½T: 10.158124,-67.998031"
1,[],RT @HELIODOPTERO: Expresidente Uribe perdió tu...,atratenobis,"Cali, Colombia"
2,[],"RT @fdbedout: Hoy con 4.213 casos, Colombia oc...",FelipeMarti5,"Bogotá, D.C., Colombia"
3,[],Fortalecer para callar:las propuestas d Colomb...,Reiniciarco,"Bogotá, Colombia"
4,[],RT @LatinasTrans: •FIND A HOT TRANS LATINA•\r\...,tello_guimel7,NaN


In [69]:
#Mostrando todo
tweetsRealTime

,hashtags,text,user,location
0,['ENPolítica'],RT @ElNacionalWeb: Falleció el diputado Hernán...,roswitha76,"ï¿½T: 10.158124,-67.998031"
1,[],RT @HELIODOPTERO: Expresidente Uribe perdió tu...,atratenobis,"Cali, Colombia"
2,[],"RT @fdbedout: Hoy con 4.213 casos, Colombia oc...",FelipeMarti5,"Bogotá, D.C., Colombia"
3,[],Fortalecer para callar:las propuestas d Colomb...,Reiniciarco,"Bogotá, Colombia"
4,[],RT @LatinasTrans: •FIND A HOT TRANS LATINA•\r\...,tello_guimel7,NaN
...,...,...,...,...
75,[],Un saludito de buenas noches a toda esa parran...,DrNickolaz,NaN
76,[],"@Herobust My mom came from Colombia, my dad ca...",Sarahtzuc,"San Antonio, TX"
77,[],@MargaritaRepo Apoyo la propuesta porque viene...,kukylmrv,NaN
78,[],En que otro País llueve tanta mierda como en C...,H____P,NaN


In [52]:
from collections import Counter
import ast

tweets = pd.read_csv("work/slides/data/tweets.csv")
tweets

,hashtags,text,user,location
0,['ENPolítica'],RT @ElNacionalWeb: Falleció el diputado Hernán...,roswitha76,"ï¿½T: 10.158124,-67.998031"
1,[],RT @HELIODOPTERO: Expresidente Uribe perdió tu...,atratenobis,"Cali, Colombia"
2,[],"RT @fdbedout: Hoy con 4.213 casos, Colombia oc...",FelipeMarti5,"Bogotá, D.C., Colombia"
3,[],Fortalecer para callar:las propuestas d Colomb...,Reiniciarco,"Bogotá, Colombia"
4,[],RT @LatinasTrans: •FIND A HOT TRANS LATINA•\r\...,tello_guimel7,NaN
5,[],RT @negrogaviria: Yo no creo que @petrogustavo...,andresf_lombo,NaN
6,[],RT @ecuanimusjustus: Esto no le gustará a la i...,yanioosl,NaN
7,[],"En todo caso, será levanten los cascos",SOSinformacion,Venezuela
8,[],"RT @ernestosamperp: En Colombia, como en mucha...",DanielB23693560,NaN
9,['DaríoMonsalve'],RT @RUBYELENA1: Esta oveja negra #DaríoMonsalv...,goham_7777,Bogota


In [22]:
# Instalamos geopy
!pip install geopy

In [23]:
# Instalamos gmplot
!pip install gmplot

In [38]:
df = pd.DataFrame(tweets)

df

,hashtags,text,user,location
0,['ENPolítica'],RT @ElNacionalWeb: Falleció el diputado Hernán...,roswitha76,"ï¿½T: 10.158124,-67.998031"
1,[],RT @HELIODOPTERO: Expresidente Uribe perdió tu...,atratenobis,"Cali, Colombia"
2,[],"RT @fdbedout: Hoy con 4.213 casos, Colombia oc...",FelipeMarti5,"Bogotá, D.C., Colombia"
3,[],Fortalecer para callar:las propuestas d Colomb...,Reiniciarco,"Bogotá, Colombia"
4,[],RT @LatinasTrans: •FIND A HOT TRANS LATINA•\r\...,tello_guimel7,NaN
5,[],RT @negrogaviria: Yo no creo que @petrogustavo...,andresf_lombo,NaN
6,[],RT @ecuanimusjustus: Esto no le gustará a la i...,yanioosl,NaN
7,[],"En todo caso, será levanten los cascos",SOSinformacion,Venezuela
8,[],"RT @ernestosamperp: En Colombia, como en mucha...",DanielB23693560,NaN
9,['DaríoMonsalve'],RT @RUBYELENA1: Esta oveja negra #DaríoMonsalv...,goham_7777,Bogota


In [63]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim(user_agent="BD-Project-JJ")

no_location = 0

#revisar todos los tweets y agregar ubicaciones 
coordinates =  {'latitude': [], 'longitude': []}
for count, user_location in enumerate(tweets.location):
    try: 
        location = geolocator.geocode(user_location)

        #si se encuentra coordenadas 
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)

        else:
            no_location += 1
            coordinates['latitude'].append(0)
            coordinates['longitude'].append(0)

    except:
        #si hay demasiads solicitudes de conexion
        pass



In [64]:
#coordinates['latitude']
no_location
#tweets

4

In [66]:
data = tweets

data['latitude'] = coordinates['latitude']
data['longitude'] = coordinates['longitude']

data.to_csv('work/slides/data/tweetCoordinates.csv', index = False, header=True)

data


,hashtags,text,user,location,latitude,longitude
0,['ENPolítica'],RT @ElNacionalWeb: Falleció el diputado Hernán...,roswitha76,"ï¿½T: 10.158124,-67.998031",0.000000,0.000000
1,[],RT @HELIODOPTERO: Expresidente Uribe perdió tu...,atratenobis,"Cali, Colombia",3.451792,-76.532494
2,[],"RT @fdbedout: Hoy con 4.213 casos, Colombia oc...",FelipeMarti5,"Bogotá, D.C., Colombia",9.341263,-74.495072
3,[],Fortalecer para callar:las propuestas d Colomb...,Reiniciarco,"Bogotá, Colombia",4.598080,-74.076044
4,[],RT @LatinasTrans: •FIND A HOT TRANS LATINA•\r\...,tello_guimel7,NaN,46.314475,11.048029
5,[],RT @negrogaviria: Yo no creo que @petrogustavo...,andresf_lombo,NaN,46.314475,11.048029
6,[],RT @ecuanimusjustus: Esto no le gustará a la i...,yanioosl,NaN,46.314475,11.048029
7,[],"En todo caso, será levanten los cascos",SOSinformacion,Venezuela,8.001871,-66.110932
8,[],"RT @ernestosamperp: En Colombia, como en mucha...",DanielB23693560,NaN,46.314475,11.048029
9,['DaríoMonsalve'],RT @RUBYELENA1: Esta oveja negra #DaríoMonsalv...,goham_7777,Bogota,4.598080,-74.076044


In [30]:
# instancia
gmap = gmplot.GoogleMapPlotter(30,0,3)

#insertar puntos en el mapa pasando una lista de coordenadas
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

#Guardar el mapa
gmap.draw('work/slides/heatmap.html')